### Merge Upstream Downstream with FAO names 

* Purpose of script: Create a shapefile and csv file with both the upstream / downstream relation and the FAO basin names
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20170829

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

Y2017M09D26 UTC 15:29


In [2]:
S3_INPUT_PATH_FAO ="s3://wri-projects/Aqueduct30/processData/Y2017M08D25_RH_spatial_join_FAONames_V01/output/"
S3_INPUT_PATH_DOWNSTREAM = "s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output/"
S3_INPUT_PATH_HYBAS = "s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/"

INPUT_FILE_NAME_FAO = "hybas_lev06_v1c_merged_fiona_withFAO_V01.csv"
INPUT_FILE_NAME_DOWNSTREAM = "hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv"
INPUT_FILE_NAME_HYBAS = "hybas_lev06_v1c_merged_fiona_V01.shp"

EC2_INPUT_PATH = "/volumes/data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/"
EC2_OUTPUT_PATH = "/volumes/data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

OUTPUT_FILE_NAME = "hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01"

S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

In [3]:
!rm -r {EC2_INPUT_PATH}
!rm -r {EC2_OUTPUT_PATH}

!mkdir -p {EC2_INPUT_PATH}
!mkdir -p {EC2_OUTPUT_PATH}

In [4]:
!aws s3 cp {S3_INPUT_PATH_FAO} {EC2_INPUT_PATH} --recursive 

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D25_RH_spatial_join_FAONames_V01/output/hybas_lev06_v1c_merged_fiona_withFAO_V01.csv to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev06_v1c_merged_fiona_withFAO_V01.csv


In [5]:
!aws s3 cp {S3_INPUT_PATH_DOWNSTREAM} {EC2_INPUT_PATH} --recursive 

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv


In [6]:
!aws s3 cp {S3_INPUT_PATH_HYBAS} {EC2_INPUT_PATH} --recursive --exclude *.tif

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev00_v1c_merged_fiona_V01.cpg to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev00_v1c_merged_fiona_V01.cpg
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev00_v1c_merged_fiona_V01.prj to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev00_v1c_merged_fiona_V01.prj
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merged_fiona_V01.prj to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev06_v1c_merged_fiona_V01.prj
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merged_fiona_V01.cpg to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev06_v1c_merged_fiona_V01.cpg
download: s3://wri-projects/Aqueduct30/processDa

In [7]:
import os
if 'GDAL_DATA' not in os.environ:
    os.environ['GDAL_DATA'] = r'/usr/share/gdal/2.1'
from osgeo import gdal,ogr,osr
'GDAL_DATA' in os.environ
# If false, the GDAL_DATA variable is set incorrectly. You need this variable to obtain the spatial reference
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import time
%matplotlib notebook

In [8]:
dfFAO = pd.read_csv(os.path.join(EC2_INPUT_PATH,INPUT_FILE_NAME_FAO))

In [9]:
dfFAO.head()

,PFAF_ID,SUB_NAME,MAJ_NAME,FAOid_copy
0,111011,['Wadi El Naqa'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190312']
1,111012,['Egyptian east coast'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190313']
2,111013,['Egyptian east coast'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190313']
3,111014,['Egyptian east coast'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190313']
4,111015,['Egyptian east coast'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190313']


In [10]:
dfDownstream = pd.read_csv(os.path.join(EC2_INPUT_PATH,INPUT_FILE_NAME_DOWNSTREAM))

In [11]:
dfDownstream.head()

,HYBAS_ID2,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,...,COAST,ORDER,SORT,Upstream_HYBAS_IDs,Upstream_PFAF_IDs,Downstream_HYBAS_IDs,Downstream_PFAF_IDs,NEXT_SINK_PFAF,Basin_HYBAS_IDs,Basin_PFAF_IDs
0,6060000010,0,6060000010,0,6060000010,6060000010,0.0,0.0,4317.4,4317.4,...,1,0,1,[],[],[],[],611001.0,[6060000010],[611001]
1,6060000200,1,6060000200,0,6060000200,6060000200,0.0,0.0,35995.5,35996.7,...,0,1,2,[],[],[],[],611002.0,[6060000200],[611002]
2,6060000210,2,6060000210,0,6060000210,6060000210,0.0,0.0,443.9,443.9,...,1,0,3,[],[],[],[],611003.0,[6060000210],[611003]
3,6060000240,3,6060000240,0,6060000240,6060000240,0.0,0.0,2186.3,2186.3,...,0,1,4,[],[],[],[],611004.0,[6060000240],[611004]
4,6060000250,4,6060000250,0,6060000250,6060000250,0.0,0.0,6533.8,6533.8,...,1,0,5,[],[],[],[],611005.0,[6060000250],[611005]


In [12]:
gdfHybas = gpd.read_file(os.path.join(EC2_INPUT_PATH,INPUT_FILE_NAME_HYBAS))

In [13]:
dfHybas = gdfHybas.drop('geometry',1)

In [14]:
dfSimple = pd.DataFrame(dfHybas["PFAF_ID"])

In [15]:
dfSimple = dfSimple.set_index("PFAF_ID", drop=False)

In [16]:
gdfHybas2 = gdfHybas.set_index("PFAF_ID", drop=False)

In [17]:
gdfHybasSimple = gpd.GeoDataFrame(dfSimple, geometry=gdfHybas2.geometry)

In [18]:
dfDownstream.head()

,HYBAS_ID2,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,...,COAST,ORDER,SORT,Upstream_HYBAS_IDs,Upstream_PFAF_IDs,Downstream_HYBAS_IDs,Downstream_PFAF_IDs,NEXT_SINK_PFAF,Basin_HYBAS_IDs,Basin_PFAF_IDs
0,6060000010,0,6060000010,0,6060000010,6060000010,0.0,0.0,4317.4,4317.4,...,1,0,1,[],[],[],[],611001.0,[6060000010],[611001]
1,6060000200,1,6060000200,0,6060000200,6060000200,0.0,0.0,35995.5,35996.7,...,0,1,2,[],[],[],[],611002.0,[6060000200],[611002]
2,6060000210,2,6060000210,0,6060000210,6060000210,0.0,0.0,443.9,443.9,...,1,0,3,[],[],[],[],611003.0,[6060000210],[611003]
3,6060000240,3,6060000240,0,6060000240,6060000240,0.0,0.0,2186.3,2186.3,...,0,1,4,[],[],[],[],611004.0,[6060000240],[611004]
4,6060000250,4,6060000250,0,6060000250,6060000250,0.0,0.0,6533.8,6533.8,...,1,0,5,[],[],[],[],611005.0,[6060000250],[611005]


In [19]:
dfOut = dfSimple.merge(dfDownstream, on='PFAF_ID',how="outer")

In [20]:
dfOut = dfOut.merge(dfFAO,on='PFAF_ID',how="outer")

In [ ]:
gdfHybasSimple.to_file(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILE_NAME+".shp"))

In [ ]:
dfOut.to_csv(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILE_NAME+".csv"))

In [ ]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive